In [2]:
import network  # type: ignore

class Radio:
    def __init__(self):
        self._enabled_count = 0
        self._sta = network.WLAN(network.STA_IF)
        print("Radio init", self._sta)

    def __aenter__(self):
        print("Radio aenter", self._sta)
        if self._enabled_count < 1:
            self._sta.active(True)
        self._enabled_count += 1
        return self

    def __aexit__(self, *args):
        print("Radio aexit", self._sta)
        self._enabled_count -= 1
        if self._enabled_count < 1:
            try:
                self._sta.active(False)
            except OSError:
                pass

    def scan(self):
        return "\n".join(
            [
                f"{r[0].decode():40} ch={r[2]:2} rssi={r[3]:3}dbM"
                for r in self._sta.scan()
                if r[0]
            ]
        )

    def ssids(self):
        return [r[0].decode() for r in self._sta.scan() if r[0]]
    
    async def mac_address(self) -> str:
        print("mac_address", self, self._sta)
        async with self:
            wlan_mac = self._sta.config("mac")
            return ":".join([f"{b:02x}" for b in wlan_mac])

async def main():
    a = Radio()
    async with a as manager:
        print("Inside the context")

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())

Radio init <WLAN>
Radio aenter <WLAN>


Traceback (most recent call last):
  File "<stdin>", line 50, in <module>
  File "asyncio/core.py", line 1, in run
  File "asyncio/core.py", line 1, in run_until_complete
  File "asyncio/core.py", line 1, in run_until_complete
  File "<stdin>", line 45, in main
TypeError: 'Radio' object isn't iterable


In [14]:
import network  # type: ignore

class Radio:
    def __init__(self):
        self._enabled_count = 0
        self._sta = network.WLAN(network.STA_IF)
        
    async def __aenter__(self):
        if self._enabled_count < 1:
            self._sta.active(True)
        self._enabled_count += 1
        return self

    async def __aexit__(self, exc_type, exc_value, traceback):
        self._enabled_count -= 1
        if self._enabled_count < 1:
            try:
                self._sta.active(False)
            except OSError:
                pass

    def scan(self):
        return "\n".join(
            [
                f"{r[0].decode():40} ch={r[2]:2} rssi={r[3]:3}dbM"
                for r in self._sta.scan()
                if r[0]
            ]
        )

    def ssids(self):
        return [r[0].decode() for r in self._sta.scan() if r[0]]
    
    async def mac_address(self) -> str:
        async with self:
            wlan_mac = self._sta.config("mac")
            return ":".join([f"{b:02x}" for b in wlan_mac])
        
async def main():
    radio = Radio()
    async with radio:
        print(await radio.mac_address())
        print(radio.scan())
        print(radio.ssids())

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())

34:85:18:77:89:6c
R                                        ch= 6 rssi=-50dbM
TPA                                      ch= 1 rssi=-51dbM
TPA-Secure                               ch= 1 rssi=-51dbM
ATT-Router                               ch=11 rssi=-62dbM
TPA-Secure                               ch=11 rssi=-67dbM
TPA                                      ch=11 rssi=-67dbM
TPA-R                                    ch= 6 rssi=-68dbM
hug2g246832                              ch= 6 rssi=-79dbM
['R', 'TPA', 'TPA-Secure', 'ATT-Router', 'TPA-Secure', 'TPA', 'TPA-R', 'ATT552']



In [5]:
with open('app/wifi/__init__.py', 'r') as f:
    print(f.read())

from util import is_micropython

class WifiException2(Exception):
    pass

print("-"*100, "Using micropython wifi", is_micropython())

if is_micropython():
    print("Using micropython wifi")
    from wifi.micropython_wifi import Radio, Wifi
else:
    from wifi.cpython_wifi import Radio, Wifi



In [18]:
%rsync

from app.wifi import Radio, Wifi

async def main():
    radio = Radio()
    async with radio:
        print(await radio.mac_address())

    wifi = Wifi()
    async with wifi:
        print(await wifi.ip)

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())

Local and remote directories match
34:85:18:77:89:6c


Traceback (most recent call last):
  File "<stdin>", line 14, in <module>
  File "asyncio/core.py", line 1, in run
  File "asyncio/core.py", line 1, in run_until_complete
  File "asyncio/core.py", line 1, in run_until_complete
  File "<stdin>", line 9, in main
  File "app/wifi/micropython_wifi.py", line 77, in __aenter__
ImportError: no module named 'app.config'
